In [1]:
import piff
import numpy as np
import matplotlib.pyplot as plt
import pickle
import copy
import time
import cProfile
import pstats
import jax
from jax import config
# if want to use float64 for jax, default will be float32 with jax
# faster with float32...
# config.update("jax_enable_x64", True)

In [2]:
# load data from a single ccd from subaru. Use the exact same input
# of what is coming in Piff in LSST stack (in meas_extensions_piff)

dic = pickle.load(open('piff_input_2024-06-24_13-43-22.pkl', 'rb'))

stars = copy.deepcopy(dic['stars'])
wcs = copy.deepcopy(dic['wcs'])
pointing = copy.deepcopy(dic['pointing'])
piffConfig = copy.deepcopy(dic['piffConfig'])

# run piff with current config in LSST stack

print(piffConfig)
logger = piff.config.setup_logger(verbose=2)
piffResult = piff.PSF.process(piffConfig)

def my_function():
    piffResult.fit(stars, wcs, pointing, logger=logger)

start = time.time()
cProfile.run('my_function()', 'profile_data')
end = time.time()

current_time = end - start

print('Total run time: ' + str(current_time))

p = pstats.Stats('profile_data')
p.strip_dirs().sort_stats('cumulative').print_stats(20)

Iteration 1: Fitting 118 stars


{'type': 'Simple', 'model': {'type': 'PixelGrid', 'scale': 0.16839226517851116, 'size': 25, 'interp': 'Lanczos(11)'}, 'interp': {'type': 'BasisPolynomial', 'order': 2}, 'outliers': {'type': 'Chisq', 'nsigma': 4.0, 'max_remove': 0.05}}


nq = 3750
Beginning solution of matrix size (3750, 3750)
             Total chisq = 64075.13 / 72029 dof
Iteration 2: Fitting 118 stars
nq = 3750
Beginning solution of matrix size (3750, 3750)
Checking 118 stars for outliers
Minimum dof = 497 with thresh = 627.063937 * 1.000000 = 627.063937
Maximum dof = 622 with thresh = 766.399073 * 1.000000 = 766.399073
Found 9 stars with chisq > thresh
             Removed 6 outliers
             Total chisq = 44329.60 / 68419 dof
Iteration 3: Fitting 112 stars
nq = 3750
Beginning solution of matrix size (3750, 3750)
Checking 112 stars for outliers
Minimum dof = 497 with thresh = 627.063937 * 1.000000 = 627.063937
Maximum dof = 622 with thresh = 766.399073 * 1.000000 = 766.399073
Found 3 stars with chisq > thresh
             Removed 3 outliers
             Total chisq = 41877.24 / 66603 dof
Iteration 4: Fitting 109 stars
nq = 3750
Beginning solution of matrix size (3750, 3750)
Checking 109 stars for outliers
Minimum dof = 497 with thresh = 627.063

Total run time: 16.07547092437744
Mon Jul  8 09:22:43 2024    profile_data

         3117396 function calls (2970938 primitive calls) in 16.073 seconds

   Ordered by: cumulative time
   List reduced from 521 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   16.073   16.073 {built-in method builtins.exec}
        1    0.000    0.000   16.073   16.073 <string>:1(<module>)
        1    0.009    0.009   16.073   16.073 2967559449.py:17(my_function)
        1    0.139    0.139   16.065   16.065 psf.py:369(fit)
        4    0.030    0.007   14.995    3.749 simplepsf.py:163(single_iteration)
        4    0.011    0.003   13.483    3.371 basis_interp.py:138(solve)
        4   10.824    2.706   13.473    3.368 basis_interp.py:288(_solve_direct)
        4    1.408    0.352    1.408    0.352 _basic.py:52(solve)
      457    0.741    0.002    1.317    0.003 pixelgrid.py:226(chisq)
     2330    1.183    0.001    1.1

In [3]:
# load data from a single ccd from subaru. Use the exact same input
# of what is coming in Piff in LSST stack (in meas_extensions_piff)
# reload to here to make sure start from the same starting point.

dic = pickle.load(open('piff_input_2024-06-24_13-43-22.pkl', 'rb'))

stars = copy.deepcopy(dic['stars'])
wcs = copy.deepcopy(dic['wcs'])
pointing = copy.deepcopy(dic['pointing'])
piffConfig = copy.deepcopy(dic['piffConfig'])

# enable jax just in some part of basis interp (mostly in _solve_direct)

jaxPiffConfig = copy.deepcopy(piffConfig)
jaxPiffConfig['interp'].update({'use_jax': True})
print(jaxPiffConfig)

logger = piff.config.setup_logger(verbose=2)
piffResult = piff.PSF.process(jaxPiffConfig)

def my_function():
    piffResult.fit(stars, wcs, pointing, logger=logger)

start = time.time()
cProfile.run('my_function()', 'profile_data')
end = time.time()

jax_time = end - start

print('Total run time: ' + str(jax_time))

p = pstats.Stats('profile_data')
p.strip_dirs().sort_stats('cumulative').print_stats(20)

Iteration 1: Fitting 118 stars


{'type': 'Simple', 'model': {'type': 'PixelGrid', 'scale': 0.16839226517851116, 'size': 25, 'interp': 'Lanczos(11)'}, 'interp': {'type': 'BasisPolynomial', 'order': 2, 'use_jax': True}, 'outliers': {'type': 'Chisq', 'nsigma': 4.0, 'max_remove': 0.05}}


nq = 3750
Beginning solution of matrix size (3750, 3750)
             Total chisq = 64075.13 / 72029 dof
Iteration 2: Fitting 118 stars
nq = 3750
Beginning solution of matrix size (3750, 3750)
Checking 118 stars for outliers
Minimum dof = 497 with thresh = 627.063937 * 1.000000 = 627.063937
Maximum dof = 622 with thresh = 766.399073 * 1.000000 = 766.399073
Found 9 stars with chisq > thresh
             Removed 6 outliers
             Total chisq = 44329.60 / 68419 dof
Iteration 3: Fitting 112 stars
nq = 3750
Beginning solution of matrix size (3750, 3750)
Checking 112 stars for outliers
Minimum dof = 497 with thresh = 627.063937 * 1.000000 = 627.063937
Maximum dof = 622 with thresh = 766.399073 * 1.000000 = 766.399073
Found 3 stars with chisq > thresh
             Removed 3 outliers
             Total chisq = 41877.24 / 66603 dof
Iteration 4: Fitting 109 stars
nq = 3750
Beginning solution of matrix size (3750, 3750)
Checking 109 stars for outliers
Minimum dof = 497 with thresh = 627.063

Total run time: 7.474876880645752
Mon Jul  8 09:23:03 2024    profile_data

         3267806 function calls (3120099 primitive calls) in 7.464 seconds

   Ordered by: cumulative time
   List reduced from 2213 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      9/1    0.000    0.000    7.465    7.465 {built-in method builtins.exec}
        1    0.000    0.000    7.465    7.465 <string>:1(<module>)
        1    0.013    0.013    7.465    7.465 1833065479.py:20(my_function)
        1    0.158    0.158    7.451    7.451 psf.py:369(fit)
        4    0.072    0.018    6.360    1.590 simplepsf.py:163(single_iteration)
        4    0.031    0.008    4.762    1.191 basis_interp.py:138(solve)
        4    1.091    0.273    4.731    1.183 basis_interp.py:288(_solve_direct)
   144/76    0.000    0.000    2.529    0.033 core.py:389(bind_with_trace)
     34/8    0.000    0.000    2.528    0.316 traceback_util.py:175(reraise_with_filtered_trace

In [4]:
print(current_time / jax_time)

2.1506000942972974
